In [1]:
import h5py
import numpy as np
import tensorflow as tf

In [2]:
def load_data(hdf5_file, dataset_name_images, dataset_name_labels):
    with h5py.File(hdf5_file, 'r') as hf:
        data = np.array(hf[dataset_name_images])
        labels = np.array(hf[dataset_name_labels])
    return data, labels

# Loading model and Testing accuracy

In [3]:
CLCM_loaded_model = tf.keras.models.load_model('Trained Model/CLCM_model.keras')
LEC_loaded_model = tf.keras.models.load_model('Trained Model/LEC_model_student_with_kd.keras')
MobileNetV3_S_loaded_model = tf.keras.models.load_model('Trained Model/MobileNetV3_small_model.keras')

TypeError: Could not locate class 'CustomDistillationModel'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'CustomDistillationModel', 'config': {'trainable': True, 'dtype': 'float32'}, 'registered_name': 'CustomDistillationModel', 'build_config': {'input_shape': [None, 224, 224, 3]}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 9.999999747378752e-06, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 'registered_name': None}, 'loss': 'sparse_categorical_crossentropy', 'loss_weights': None, 'metrics': ['accuracy'], 'weighted_metrics': None, 'run_eagerly': False, 'steps_per_ex

In [ ]:
X_fer_test, y_fer_test = load_data('output files/fer2013_testing.h5', 'images', 'labels')
X_rafdb_test, y_rafdb_test = load_data('output files/rafdb_testing.h5', 'images', 'labels')
X_ck_test, y_ck_test = load_data('output files/rafdb_testing.h5', 'images', 'labels')

## Assessing Accuracy on FER-2013

In [ ]:
score = CLCM_loaded_model.evaluate(X_fer_test, y_fer_test, verbose=1)
score = LEC_loaded_model.evaluate(X_fer_test, y_fer_test, verbose=1)
score = MobileNetV3_S_loaded_model.evaluate(X_fer_test, y_fer_test, verbose=1)

## Assessing Accuracy on RAF-DB

In [ ]:
score = CLCM_loaded_model.evaluate(X_rafdb_test, y_rafdb_test, verbose=1)
score = LEC_loaded_model.evaluate(X_rafdb_test, y_rafdb_test, verbose=1)
score = MobileNetV3_S_loaded_model.evaluate(X_rafdb_test, y_rafdb_test, verbose=1)

## Assessing Accuracy on CK+

In [ ]:
score = CLCM_loaded_model.evaluate(X_ck_test, y_ck_test, verbose=1)
score = LEC_loaded_model.evaluate(X_ck_test, y_ck_test, verbose=1)
score = MobileNetV3_S_loaded_model.evaluate(X_ck_test, y_ck_test, verbose=1)